In [112]:
import pandas as pd
import numpy as np
from collections import Counter, defaultdict, deque
import re
import math
from itertools import combinations
# pd.options.display.float_format = "{:,.4f}".format

# import matplotlib.pyplot as plt
# import seaborn as sns

# import statsmodels.api as sm
# from scipy.stats import norm
# from sklearn.linear_model import LinearRegression
# from arch import arch_model

# import warnings
# warnings.filterwarnings("ignore")

# import importlib
# import utils as ut
# importlib.reload(ut)

--- Day 1: Historian Hysteria ---

In [4]:
file_path = './data/input_1.txt'
data = np.loadtxt(file_path, dtype=int)

left, right = np.sort(data[:, 0]), np.sort(data[:, 1])

np.sum(np.abs(left - right))

np.int64(2086478)

In [5]:
counter = Counter(right)
sum(counter[l] * l for l in left)

np.int64(24941624)

--- Day 2: Red-Nosed Reports ---

In [26]:
file_path = './data/input_2.txt'
with open(file_path, 'r') as file:
    data = [list(map(int, line.split())) for line in file]

def is_safe(row):
    differences = [row[i] - row[i - 1] for i in range(1, len(row))]
    return all(1 <= diff <= 3 for diff in differences) or all(-3 <= diff <= -1 for diff in differences)

sum(is_safe(row) for row in data)

483

In [31]:
def is_removable_safe(row):
    if is_safe(row):
        return True
    for i in range(len(row)):
        new_row = row[:i] + row[i + 1:]
        if is_safe(new_row):
            return True
    return False

sum(is_removable_safe(row) for row in data)

528

--- Day 3: Mull It Over ---

In [10]:
file_path = './data/input_3.txt'
with open(file_path, 'r') as file:
    data = file.read()

pattern = r'mul\((\d+),(\d+)\)'
matches = re.findall(pattern, data)
sum(int(a) * int(b) for a, b in matches)

175615763

In [22]:
mul_pattern = r"mul\((\d+),(\d+)\)"
do_pattern = r"do\(\)"
dont_pattern = r"don't\(\)"

pattern = f"{mul_pattern}|{do_pattern}|{dont_pattern}"

matches = re.finditer(pattern, data)

mul_enabled = True
total_sum = 0

for match in matches:
    if match.group().startswith('do()'):
        mul_enabled = True
    elif match.group().startswith("don't()"):
        mul_enabled = False
    elif mul_enabled and match.group().startswith("mul("):
        a, b = map(int, match.groups())
        total_sum += a * b

total_sum

74361272

--- Day 4: Ceres Search ---

In [62]:
file_path = './data/input_4.txt'
with open(file_path, 'r') as file:
    data = file.read().strip()
lines = data.splitlines()

In [72]:
def collect_all_directions(grid):
    R, C = len(grid), len(grid[0])
    directions = []
    directions.extend(grid)
    directions.extend([row[::-1] for row in grid])
    directions.extend([''.join([grid[r][c] for r in range(R)]) for c in range(C)])
    directions.extend([''.join([grid[R - 1 - r][c] for r in range(R)]) for c in range(C)])

    for d in range(-R + 1, C):
        directions.append(''.join([grid[r][r - d] for r in range(max(0, d), min(R, C + d))]))
    for d in range(-R + 1, C):
        directions.append(''.join([grid[R - 1 - r][r - d] for r in range(max(0, d), min(R, C + d))]))
    for d in range(-R + 1, C):
        directions.append(''.join([grid[r][C - 1 - (r - d)] for r in range(max(0, d), min(R, C + d))]))
    for d in range(-R + 1, C):
        directions.append(''.join([grid[R - 1 - r][C - 1 - (r - d)] for r in range(max(0, d), min(R, C + d))]))

    return directions

directions = collect_all_directions(lines)
sum(direction.count('XMAS') for direction in directions)

2427

In [77]:
def count_X_MAS(grid):
    res = 0
    for r in range(1, len(grid) - 1):
        for c in range(1, len(grid[0]) - 1):
            if grid[r][c] == 'A':
                if set([grid[r - 1][c - 1], grid[r + 1][c + 1]]) == {'M', 'S'} and set([grid[r - 1][c + 1], grid[r + 1][c - 1]]) == {'M', 'S'}:
                    res += 1

    return res
count_X_MAS(lines)

1900

--- Day 5: Print Queue ---

In [48]:
file_path = './data/input_5.txt'
rules, pages = [], []

with open(file_path, 'r') as file:
    for line in file:
        line = line.strip()
        if '|' in line:
            rules.append(list(map(int, line.split('|'))) )
        elif ',' in line:
            pages.append(list(map(int, line.split(','))) )

In [49]:
rules_dict = defaultdict(list)
for a, b in rules:
    rules_dict[b].append(a)

def valid_middle(numbers):
    seen, forbidden = set(), set()
    for number in numbers:
        if (number in forbidden) and (number not in seen):
            return 0
        seen.add(number)
        forbidden.update(rules_dict[number])

    return numbers[len(numbers) // 2]

sum(valid_middle(numbers) for numbers in pages)

5087

In [64]:
def reorder_invalid_middle(numbers):
    seen, forbidden = set(), set()
    mid_ind = len(numbers) // 2
    for number in numbers:
        if (number in forbidden) and (number not in seen):
            reordered = sorted(numbers, key=lambda x: len([y for y in rules_dict[x] if y in numbers]))
            return reordered[mid_ind]
        seen.add(number)
        forbidden.update(rules_dict[number])
    return 0

sum(reorder_invalid_middle(numbers) for numbers in pages)

4971

--- Day 6: Guard Gallivant ---

In [4]:
file_path = './data/input_6.txt'
with open(file_path, 'r') as file:
    data = file.read().strip()
lines = data.splitlines()

In [16]:
R, C = len(lines), len(lines[0])
dirs_dict = {(-1, 0): (0, 1),
             (0, 1): (1, 0),
             (1, 0): (0, -1),
             (0, -1): (-1, 0)}

start_r, start_c = [(r, c) for r in range(R) for c in range(C) if lines[r][c] == '^'][0]
start_dir = (-1, 0)

def simulate_path():
    visited = np.zeros((R, C), dtype=int)
    r, c = start_r, start_c
    dr, dc = start_dir

    while 0 <= r < R and 0 <= c < C:
        visited[r][c] = 1
        if lines[r + dr][c + dc] == '#':
            dr, dc = dirs_dict[(dr, dc)]
        r += dr
        c += dc

    return np.sum(visited)

simulate_path()

np.int64(4883)

In [55]:
def simulate_with_obstruction(block_r, block_c):
    """Cycle is detected not just position, but also direction"""
    visited = set()
    r, c = start_r, start_c
    d = 0 # 0: up, 1: right, 2: down, 3: left

    while 0 <= r < R and 0 <= c < C:
        # print(visited)
        if (r, c, d) in visited:
            return True
        visited.add((r, c, d))
        dr, dc = [(-1, 0), (0, 1), (1, 0), (0, -1)][d]
        next_r, next_c = r + dr, c + dc
        
        if not (0 <= next_r < R and 0 <= next_c < C):
            break

        if lines[next_r][next_c] == '#' or (next_r, next_c) == (block_r, block_c):
            d = (d + 1) % 4
        else:
            r, c = next_r, next_c

    return False

In [56]:
sum(simulate_with_obstruction(r, c) for r in range(R) for c in range(C) 
    if ((r, c) != (start_r, start_c) and lines[r][c] == '.'))

1655

--- Day 7: Bridge Repair ---

In [76]:
file_path = './data/input_7.txt'
results = []
operands = []
with open(file_path, 'r') as file:
    for line in file:
        left, right = line.strip().split(':')
        results.append(int(left))
        operands.append(list(map(int, right.strip().split())))

In [ ]:
def valid_expression(operands, results):
    n = len(operands)
    acc = deque([operands[0]])
    for i in range(1, n):
        for j in range(len(acc)):
            element = acc.popleft() 
            acc.append(element + operands[i])
            acc.append(element * operands[i])
    return results in acc

sum(result for operand, result in zip(operands, results) if valid_expression(operand, result))

882304362421

In [96]:
def valid_expression_concat(operands, results):
    n = len(operands)
    acc = deque([operands[0]])
    for i in range(1, n):
        for _ in range(len(acc)):
            element = acc.popleft() 
            acc.append(element + operands[i])
            acc.append(element * operands[i])
            acc.append(int(str(element) + str(operands[i])))
            
    return results in acc

sum(result for operand, result in zip(operands, results) if valid_expression_concat(operand, result))

145149066755184

--- Day 8: Resonant Collinearity ---

In [4]:
file_path = './data/input_8.txt'
with open(file_path, 'r') as file:
    data = file.read().strip()
grid = data.splitlines()

In [5]:
R, C = len(grid), len(grid[0])
char_loc = defaultdict(list)
for r in range(R):
    for c in range(C):
        if grid[r][c] == '.':
            continue
        char_loc[grid[r][c]].append((r, c))

def get_antinode(a, b):
    res = []
    ax, ay = a
    bx, by = b
    if 0 <= 2 * bx - ax < R and 0 <= 2 * by - ay < C:
        res.append((2 * bx - ax, 2 * by - ay))
    if 0 <= 2 * ax - bx < R and 0 <= 2 * ay - by < C:
        res.append((2 * ax - bx, 2 * ay - by))
    return res

antinodes = set()
for k, v in char_loc.items():
    for a, b in combinations(v, 2):
        antinodes.update(get_antinode(a, b))
len(antinodes)

357

In [16]:
def get_antinode_inline(a, b):
    res = []
    ax, ay = a
    bx, by = b
    dx, dy = ax - bx, ay - by

    x, y = ax, ay
    while True:
        if not (0 <= x + dx < R and 0 <= y + dy < C):
            break
        else:
            x += dx
            y += dy
            res.append((x, y))

    x, y = bx, by
    while True:
        if not (0 <= x - dx < R and 0 <= y - dy < C):
            break
        else:
            x -= dx
            y -= dy
            res.append((x, y))

    return res

antinodes = set()
for k, v in char_loc.items():
    antinodes.update(v)
    for a, b in combinations(v, 2):
        antinodes.update(get_antinode_inline(a, b))
len(antinodes)

1266

--- Day 9: Disk Fragmenter ---

In [3]:
file_path = './data/input_9.txt'
with open(file_path, 'r') as file:
    data = file.read().strip()

In [65]:
def parse_and_compact(data):
    num = 0
    s = []
    for i, l in enumerate(data):
        if i % 2 == 0:
            s.extend([num] * int(l))
            num += 1
        else:
            s.extend(['.'] * int(l))

    l, r = 0, len(s) - 1
    while l < r:
        if s[l] != '.':
            l += 1
            continue
        if s[r] == '.':
            r -= 1
            continue
        s[l], s[r] = s[r], s[l]
    
    return s

s = parse_and_compact(data)
sum(i * l for i, l in enumerate(s) if l != '.')

6349606724455

In [6]:
def parse_and_compact_whole(data):
    # s is a list of blocks, each block is a file id or '.'
    files = deque([])
    spaces = deque([])
    pos = 0
    file_id = 0
    res = []
    for i, c in enumerate(data):
        if i % 2 == 0:
            files.append((pos, int(c), file_id))
            res.extend([file_id] * int(c))
            pos += int(c)
            file_id += 1
        else:
            spaces.append((pos, int(c)))
            res.extend([None] * int(c))
            pos += int(c)

    for (file_pos, file_size, file_id) in reversed(files):
        for space_i, (space_pos, space_size) in enumerate(spaces):
            if (file_pos > space_pos) and (file_size <= space_size):
                for i in range(file_size):
                    assert res[file_pos + i] == file_id
                    res[file_pos + i] = None
                    res[space_pos + i] = file_id
                spaces[space_i] = (space_pos + file_size, space_size - file_size)
                break
    
    return res

res = parse_and_compact_whole(data)
print(res)
sum(i * l for i, l in enumerate(res) if l is not None)

[0, 0, 0, 9999, 9998, 9998, 9998, 9998, 9997, 9997, 9992, 1, 1, 1, 1, 1, 1, 9991, 9972, 2, 2, 2, 2, 2, 2, 9996, 9996, 9996, 9996, 9988, 9988, 3, 9995, 9995, 9995, 9995, 9995, 9995, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 9994, 9994, 9994, 9994, 9989, 9989, 9989, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 9986, 9986, 9982, 9982, 9982, 8, 8, 8, 8, 8, 8, 8, 9957, 9, 9, 9, 9, 9, 9, 9, 9993, 9993, 9993, 9993, 9993, 9993, 9993, 9952, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 9990, 9990, 9990, 9990, 9990, 9990, 9981, 9981, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 9985, 9985, 9985, 9985, 9985, 9948, 14, 14, 14, 14, 14, 14, 14, 14, 14, 9984, 9984, 9984, 9984, 15, 15, 15, 15, 9980, 9980, 9980, 16, 16, 16, 16, 16, 16, 16, 16, 9983, 9983, 9983, 9983, 9983, 9974, 9974, 9947, 17, 17, 9973, 9973, 9973, 18, 18, 18, 18, 18, 18, 18, 9942, 19, 19, 9966, 9966, 20, 20, 20, 21, 21, 21, 21, 9964, 9964, 9964, 22, 22, 22, 22, 22, 22, 22, 22, 9979, 9979, 9979, 9979, 9979, 99

6376648986651

--- Day 10: Hoof It ---

In [32]:
file_path = './data/input_10.txt'
with open(file_path, 'r') as file:
    data = file.read().strip()
grid = data.splitlines()

In [ ]:
small_grid = """89010123
78121874
87430965
96549874
45678903
32019012
01329801
10456732"""

In [33]:
def search(grid, num):
    R, C = len(grid), len(grid[0])
    res = []
    for r in range(R):
        for c in range(C):
            if grid[r][c] == num:
                res.append((r, c))
    return res

# multi-source BFS
dirs = [(0, 1), (1, 0), (0, -1), (-1, 0)]
R, C = len(grid), len(grid[0])

trail_map = {}
for i in range(9):
    cur_map = defaultdict(list)
    starts = search(grid, str(i))
    for r, c in starts:
        for dr, dc in dirs:
            nr, nc = r + dr, c + dc
            if 0 <= nr < R and 0 <= nc < C and grid[nr][nc] == str(i + 1):
                cur_map[(r, c)].append((nr, nc))
    trail_map[i] = cur_map

In [34]:
def count_trail_ends(trail_head, trail_map):
    dq = deque([trail_head])
    for level in range(9):
        for _ in range(len(dq)):
            r, c = dq.popleft()
            for nr, nc in trail_map[level].get((r, c), []):
                dq.append((nr, nc))
    return len(set(dq))

sum(count_trail_ends(trail_head, trail_map) for trail_head in trail_map[0])

733

In [35]:
def count_trails(trail_head, trail_map):
    dq = deque([trail_head])
    for level in range(9):
        for _ in range(len(dq)):
            r, c = dq.popleft()
            for nr, nc in trail_map[level].get((r, c), []):
                dq.append((nr, nc))
    return len(dq)

sum(count_trails(trail_head, trail_map) for trail_head in trail_map[0])

1514

--- Day 11: Plutonian Pebbles ---

In [24]:
file_path = './data/input_11.txt'
with open(file_path, 'r') as file:
    for line in file:
        inputs = list(map(int, line.strip().split()))

In [ ]:
small_inputs = 

In [54]:
def transform_number(number):
    s_num = str(number)
    if number == 0:
        return [1]
    if len(s_num) % 2 == 0: # even digits
        mid = len(s_num) // 2
        return [int(s_num[:mid]), int(s_num[mid:])]
    return [number * 2024]

res = inputs
for _ in range(25):
    new_res = []
    for number in res:
        new_res.extend(transform_number(number))
    res = new_res

len(res)

213625

Key Optimizations
- Reduce Memory Usage with Counts Instead of Expanding Lists
- Avoid String Conversions: arithmetic instead
- Memoization: Counter `update`. Handle dictionary change size during iteration

In [17]:
def transform_number_fast(number, count):
    if number == 0:
        return Counter({1: count})
    num_digits = len(str(number))
    if num_digits % 2 == 0: # even digits
        mid = num_digits // 2
        left = number // (10 ** mid)
        right = number % (10 ** mid)
        if left == right:
            return Counter({left: 2 * count})
        return Counter({left: count, right: count})
    return Counter({number * 2024: count})

In [26]:
counter = Counter(inputs)
for _ in range(75):
    new_counter = Counter()
    for number, count in counter.items():
        new_counter.update(transform_number_fast(number, count))
    counter = new_counter
sum(v for k, v in counter.items())

252442982856820

--- Day 12: Garden Groups ---

In [101]:
file_path = './data/input_12.txt'
with open(file_path, 'r') as file:
    data = file.read().strip()
grid = data.splitlines()

In [94]:
data = """AAAA
BBCD
BBCC
EEEC"""
small_grid = data.splitlines()

In [88]:
def bfs_find_region(grid, start, dirs, visited):
    """Perform BFS to find a connected region with the same color."""
    R, C = len(grid), len(grid[0])
    r, c = start
    color = grid[r][c]
    dq = deque([(r, c)])
    patch = set()
    edges = defaultdict(set)
    
    while dq:
        # all squares of a color
        r, c = dq.popleft()
        if visited[r][c]:
            continue
        visited[r][c] = 1
        patch.add((r, c))
        # boundary squares
        for dr, dc in dirs:
            nr, nc = r + dr, c + dc
            if 0 <= nr < R and 0 <= nc < C and grid[nr][nc] == color:
                dq.append((nr, nc))
            else:
                edges[(dr, dc)].add((r, c)) # collect up, down, left, right edges

    return patch, edges

def calculate_circumference(patch, dirs):
    """Calculate the circumference of a region."""
    circumference = 0
    for r, c in patch:
        neighbor_count = sum((r + dr, c + dc) in patch for dr, dc in dirs)
        circumference += 4 - neighbor_count
    return circumference

def get_region(grid, start, visited):
    """Get the area, circumference, and visited grid of a region."""
    dirs = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    patch, edges = bfs_find_region(grid, start, dirs, visited)
    area = len(patch)
    circumference = calculate_circumference(patch, dirs)
    return area, circumference

def tally_all_region(grid):
    """Tally the area and circumference of all regions."""
    R, C = len(grid), len(grid[0])
    dirs = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    visited = np.zeros((R, C), dtype=int)
    total_area_circum = 0

    for r in range(R):
        for c in range(C):
            if not visited[r][c]:
                area, circumference = get_region(grid, (r, c), visited)
                total_area_circum += area * circumference
    return total_area_circum

tally_all_region(grid)

1573474

In [102]:
def calculate_sides(edges, dirs):
    """Calculate the sides of a region: this algorithm enforces that
    the sides can only comes from 4 directions of boundary squares,
    and each straight line only counts for once by getting more in `seen`"""
    sides = 0
    for dir_, perims in edges.items():
        seen = set()
        for (pr, pc) in perims:
            if (pr, pc) not in seen:
                sides += 1
                dq = deque([(pr, pc)])
                while dq:
                    r, c = dq.popleft()
                    if (r, c) in seen:
                        continue
                    seen.add((r, c))
                    for dr, dc in dirs:
                        nr, nc = r + dr, c + dc
                        if (nr, nc) in perims:
                            dq.append((nr, nc))
    return sides

def get_discounted_region(grid, start, visited):
    """Get the area, circumference, and visited grid of a region."""
    dirs = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    patch, edges = bfs_find_region(grid, start, dirs, visited)
    area = len(patch)
    sides = calculate_sides(edges, dirs)
    return area, sides

def tally_all_discounted_region(grid):
    """Tally the area and circumference of all regions."""
    R, C = len(grid), len(grid[0])
    dirs = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    visited = np.zeros((R, C), dtype=int)
    total_area_circum = 0

    for r in range(R):
        for c in range(C):
            if not visited[r][c]:
                area, sides = get_discounted_region(grid, (r, c), visited)
                total_area_circum += area * sides
    return total_area_circum

tally_all_discounted_region(grid)

966476

--- Day 13: Claw Contraption ---

In [151]:
file_path = './data/input_13.txt'
with open(file_path, 'r') as file:
    data = file.read().strip()

In [138]:
data = """Button A: X+94, Y+34
Button B: X+22, Y+67
Prize: X=8400, Y=5400

Button A: X+26, Y+66
Button B: X+67, Y+21
Prize: X=12748, Y=12176

Button A: X+17, Y+86
Button B: X+84, Y+37
Prize: X=7870, Y=6450

Button A: X+69, Y+23
Button B: X+27, Y+71
Prize: X=18641, Y=10279"""

In [152]:
pattern = r"Button A: X\+(\d+), Y\+(\d+)\nButton B: X\+(\d+), Y\+(\d+)\nPrize: X=(\d+), Y=(\d+)"

parsed = []
for match in re.finditer(pattern, data):
    button_a = (int(match.group(1)), int(match.group(2)))
    button_b = (int(match.group(3)), int(match.group(4)))
    prize = (int(match.group(5)), int(match.group(6)))
    parsed.append((button_a, button_b, prize))

In [136]:
tolerance = 1e-9

result = 0
cost = np.array([3, 1])
for machine in parsed:
    (Ax, Ay), (Bx, By), (Px, Py) = machine
    move = np.array([[Ax, Bx], [Ay, By]])
    prize = np.array([Px, Py])
    sol = np.linalg.solve(move, prize)
    if all(abs(x - round(x)) < tolerance for x in sol):
        result += sol @ cost
result

np.float64(37901.0)

In [153]:
tolerance = 1e-4
offset = 10000000000000

result = 0
cost = np.array([3, 1])
for machine in parsed:
    (Ax, Ay), (Bx, By), (Px, Py) = machine
    move = np.array([[Ax, Bx], [Ay, By]])
    prize = np.array([Px + offset, Py + offset])
    sol = np.linalg.solve(move, prize)
    if all(abs(x - round(x)) < tolerance for x in sol):
        result += sol @ cost
result

np.float64(77407675412647.0)

--- Day 14: Restroom Redoubt ---

In [154]:
data = """p=0,4 v=3,-3
p=6,3 v=-1,-3
p=10,3 v=-1,2
p=2,0 v=2,-1
p=0,0 v=1,3
p=3,0 v=-2,-2
p=7,6 v=-1,-3
p=3,0 v=-1,-2
p=9,3 v=2,3
p=7,3 v=-1,2
p=2,4 v=2,-3
p=9,5 v=-3,-3"""

In [155]:
pattern = r"p=(\d+),(\d+) v=(-?\d+),(-?\d+)"

parsed = []
for match in re.finditer(pattern, data):
    p = (int(match.group(1)), int(match.group(2)))
    v = (int(match.group(3)), int(match.group(4)))
    parsed.append((p, v))

In [156]:
parsed

[((0, 4), (3, -3)),
 ((6, 3), (-1, -3)),
 ((10, 3), (-1, 2)),
 ((2, 0), (2, -1)),
 ((0, 0), (1, 3)),
 ((3, 0), (-2, -2)),
 ((7, 6), (-1, -3)),
 ((3, 0), (-1, -2)),
 ((9, 3), (2, 3)),
 ((7, 3), (-1, 2)),
 ((2, 4), (2, -3)),
 ((9, 5), (-3, -3))]